앞서 람다계산법의 문법구조(그림 \ref{fig:ChurchDeBruijn})를 소개하며
수학에서 보통 $f(x)=e$ 형태로 정의되는 함수를 람다식으로는 이름을 붙이지 않고
$\lambda x.e$ 형태의 함수요약(function abstraction)식으로 표현한다고 설명했다.
여기서는 $e_1~e_2$ 형태의 함수적용(function application)식을 포함한
람다계산법의 의미구조(그림 \ref{fig:UTLC})를 정리해 보자. 의미구조를
자세히 실펴보기 앞서 먼저 함수적용식 $e_1~e_2$의 의미를 간단히 이야기하자면,
수학에서 일반적인 표기로는 $e_1(e_2)$를 뜻한다. 즉, 함수 $f$에 인자 $v$를 넘겨
계산한 값을 나타내는 $f(v)$를 람다식으로는 괄호를 쓸 필요 없이 $f~v$로
표현할 수 있다는 말이다.

그렇다면 람다계산법에서 함수를 인자에 적용해 계산한다는 의미는 구체적으로
무엇일까? 함수적용식의 왼쪽이 명백한 함수의 형태, 즉 함수요약식일 때
함수 적용 계산 과정의 가장 핵심적인  단계를 진행할 수 있다.
즉 $(\lambda x. e)\;e_2$ 형태일 때 함수의 파라메터인 $x$에 인자 $e_2$를
넘겨 계산을 한 단계 더 진행하면 $\{x{\mapsto}e_2\}e$로 표시되는,
$e$에 자유롭게 나타나는 $x$를 $e_2$로 납치없이 바꿔친 식을 얻는다.
즉, $(\lambda x. e)\;e_2 \longmapsto \{x{\mapsto}e_2\}e$가 바로
함수 적용의 계산과정 중 가장 핵심적인 단계를 나타내는 규칙($\beta$)으로
그림 \ref{fig:UTLC}에 정리된 작은걸음 의미구조의 규칙들 중 중 가장
처음으로 나타나 있다. 이렇게 $\beta$규칙을 적용해 나감으로써
계산을 진행하는 과정을 \`$\beta$줄이기' 혹은 \`$\beta$줄임'($\beta$-reduction)이라고
하며, $\beta$규칙을 적용할 수 있는 $(\lambda x. e)\;e_2$ 형태의 식을
$\beta$규칙으로 줄어드는 식($\beta$-reducible expression)의
줄임말인 \`$\beta$줄식'($\beta$-redex)이라 부른다.
마찬가지로 그 다음에 나타나는 $\eta$규칙의 경우에도 \`$\eta$줄임'($\eta$-reduction)과
\`$\eta$줄식'($\eta$-redex)의 개념을 생각해 볼 수 있다. 당분간은 $\eta$ 규칙은
없이 나머지 규칙($\beta$,$\lambda$,$\textrm{@}_1$,$\textrm{@}_2$)만 있다고 생각하자.

\begin{align*}
e ::=~& x           & {\scriptsize\textrm{name}} \\
 \mid~& \lambda x.e & {\scriptsize\textrm{abstraction}} \\
 \mid~& e~e         & {\scriptsize\textrm{application}}
\end{align*}

$$
{\scriptstyle(\beta)}\frac{}{
~~(\lambda x.e)\;e_2 \longmapsto \{x{\mapsto}e_2\}e~~}
\qquad\qquad
{\scriptstyle(\eta)}\frac{}{
~~\lambda x.e\;x \longmapsto e~~}
({\scriptstyle x\,\notin\mathrm{fv}(e) })
$$
$$
{\scriptstyle(\lambda)}\frac{e\longmapsto e'}{
~~\lambda x.e \longmapsto \lambda x.e'~~}
\qquad
{\scriptstyle(\textrm{@}_1)}\frac{e_1\longmapsto e_1'}{
~~e_1\;e_2 \longmapsto e_1'\;e_2~~}
\quad
{\scriptstyle(\textrm{@}_2)}\frac{e_2\longmapsto e_2'}{
~~e_1\;e_2 \longmapsto e_1\;e_2'~~}
$$

방금 설명한 $\beta$규칙과 같은 계산이 하스켈 언어에서도 이루어지고 있음을 확인해 보자.
우선 이 책에서 실습용으로 작성하는 하스켈 노트북의 첫 셀은 항상 다음과 같이 시작한다.\footnote{%
불필요한 코딩 스타일 제안 메시지를 끄고 필요한 컴파일러 언어 기능 설정을 하는 내용으로,
세부사항은 책의 내용을 이해하는 데 큰 관계가 없으니 이해하려 하지 않고 넘어가도 좋다.}

In [1]:
:opt no-lint                          -- linter 끄기
{-# LANGUAGE ScopedTypeVariables #-} -- 추가 언어 기능 설정

하스켈 코드에서는 $\lambda x.e$와 같은 람다식을 `\x -> e`와 같이 표현한다.
아래와 같이 람다식으로 표현된 함수를 같이 인자에 적용해 보면
함수 몸체 $e$에 나타나는 자유로운 $x$를 넘겨받은 인자로 바꿔치기하며 $\beta$줄임처럼 계산됨을 관찰할 수 있다.

In [2]:
(\x -> [x,x,x]) 7

[7,7,7]

In [3]:
(\x -> [x,x,x]) "go"

["go","go","go"]

참고로 다음과 같이 하나의 셀에 여러 개의 식을 여러 줄에 걸쳐 작성하면
각각의 식을 실행한 결과를 출력한다.

In [4]:
(\x -> [x,x,x]) 7
(\x -> [x,x,x]) "go"

[7,7,7]

["go","go","go"]

\vspace*{1ex}

람다식의 의미구조(그림 \ref{fig:UTLC})에서 $\beta$규칙과 같이
계산 진행 단계의 핵심적인 내용을 표현하는 규칙 이외의
$\lambda$,$\textrm{@}_1$,$\textrm{@}_2$와 같은 규칙은
여러 부분으로 이루어진 복합식의 어느 부분식에서
핵심적인 계산의 내용을 진행시킬지 그 맥락을 정해준다. 
$\lambda$규칙은 함수요약식($\lambda x.e$)의 함수 몸체($e$) 부분이라는 맥락에서,
$\textrm{@}_1$와 $\textrm{@}_2$규칙은 함수적용식($e_1~e_2$)의
함수($e_1$)와 인자($e_2$) 부분이라는 맥락에서 계산을 진행시키는 규칙이다.
이렇게 의미구조에서 계산의 핵심적인 내용 자체를 정의하는 것이 아니라
복합식의 문법구조에 따라 어떤 맥락(context)에서 계산이 진행될지 정의하는
규칙을 맥락규칙(context rule)이라고 부른다.
복합식이 그 자체로는 $\beta$줄식($\beta$-redex)가 아니더라도
맥락규칙에 따라 그 안에 포함된 $\beta$줄식인 부분식을 찾아
거기에 $\beta$규칙을 적용함으로써 $\beta$줄이기 계산을 진행시킬 수 있다.

In [5]:
("Hello, Haskell", (\x -> [x,x,x]) 7, "Bye, Haskell")

("Hello, Haskell",[7,7,7],"Bye, Haskell")

In [6]:
("Hello, Haskell", (\x -> [x,x,x]) "go", "Bye, Haskell")

("Hello, Haskell",["go","go","go"],"Bye, Haskell")

하스켈 코드에서도 위와 같이 줄어드는 식이 있는 부분의 맥락을 찾아 계산함을 관찰할 수 있다.

\lipsum[1-2]